## Numerical Accuracy
**Nick Kern**
<br>
**Astro 9: Python Programming in Astronomy**
<br>
**UC Berkeley**

Reading: [Chp. 4, Computational Physics w/ Python](http://www-personal.umich.edu/~mejn/computational-physics/)

We have already explored the built-in data types in Python. In this lecture, we will explore the limits of their accuracy and ranges and see how this can affect the accuracy of basic mathemtical computations.

Floats in Python have a maximum and minimum bound on their values. If one exceeds these, Python will assume them to be `inf`, `-inf` or `0.0`. These bounds correspond to the limits of the [64-bit precision](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) of your computer. 

In [ ]:
# there is a maximum value to a float in Python
1.7e308

In [ ]:
# there is a minimum value to a float in Python
-1.7e308

In [ ]:
# there is also a minimum (positive) value
1e-323

In [ ]:
# the upper bound corresponds to 
2.0 ** 1023.99

While these bounds do exist, the ranges are so large that often we don't encounter problems in typical applications.

Integers, on the other hand, have no specific limits on their precision, which is termed [arbitrary-precision](https://en.wikipedia.org/wiki/Arbitrary-precision_arithmetic). This means that the computer will store as many digits on an `int` as you want, so long as it doesn't run out of memory. This can be highly beneficial in cases where it is imperative to a calculation to keep lots of digits. However, this often means these kinds of computations with arbitrary-precision types are much slower, because they can eat up large chunks of memory if not explicitely controlled.

In [ ]:
# we can make a really big integer
2 ** 2000

You recall from a previous homework we saw that the factorial operator ! can be written recursively as

In [ ]:
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

In [ ]:
# I can axpress factorial 200 with integers and not floats
factorial(200)

Not only do floating-point numbers have bounds on their value, they also have limits to their precision. The standard precision is 16 significant digits. Beyond 16 digits, the computer can not reliably give us information on the value of a number, so it rounds it off. This is called *round-off error* or *rounding error*. We previously saw an example of this behavior:

In [ ]:
1.0 == 0.99999999999999999

The computer cannot store the 17th digit, so it rounds it up which makes the RHS equal to 1.0. For typical applications, 16 significant digits is plenty of precision to perform a calculation to high fidelity.

One thing we need to be careful of, though, is when we want to test the equality of two floats. It is bad practice to directly test their equality as

In [ ]:
x = 3.3
if x == 3.3:
    print("It's a match!")

This is because of round-off error, and the fear is that `x` may have been manipulated in some way that made the rounding of the 17th digit go a different way, in which case `x = 3.3000000000000001`, and `x != 3.3`, even though clearly we would still want `x` to equal 3.3. Instead, we can define a tolerance level and do something like this

In [ ]:
x = 3.300000000000001
tol = 1e-12
if abs(x - 3.3) < tol:
    print("It's a match!")

In this case, even if we get a weird round-off error and `x = 3.30000000000000001`, the condition is still met because the tolerance level of $10^{-12}$ is 4 digits behind the round-off.

There are particular cases when we need to consider the effects of round-off error in our calculations, and that is when we are taking a **subtraction** of two numbers that are nearly identical. This is called **subtraction error**. Consider the subtraction of 1000000000001 from 1000000000001.2345678. Because we can only store 16 sig-figs, the latter is actually represented as 1000000000001.23, and their subtraction yields 1.23, which is only accurate to three sig-figs! All subsequent calculations that rely on this result will also be affected by its large subtraction error.

**Example: Quadratic Equations**

Given a quadratic equation

\begin{align}
ax^{2} + bx + c = 0
\end{align}

its solutions can be written as

\begin{align}
x = \frac{-b \pm \sqrt{b^{2} - 4ac}}{2a}
\end{align}

Another way to equivalently write this (by multiplying by $-b\pm\sqrt{b^{2}-4ac}$ ) is

\begin{align}
x = \frac{2c}{-b \mp \sqrt{b^{2} - 4ac}}
\end{align}

Let's write a program for both and apply them to the equation: $0.001x^{2} + 1000x + 0.001 = 0$, whose solutions (according to Wolfram Alpha) are: $x=-1.000000000001\times10^{-6}$ and $x=-999999.999999000$

In [ ]:
import numpy as np

In [ ]:
def quad_solve1(a, b, c):
    sol1 = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)
    sol2 = (-b - np.sqrt(b**2 - 4*a*c)) / (2*a)
    return sol1, sol2

def quad_solve2(a, b, c):
    sol1 = 2*c / (-b - np.sqrt(b**2 - 4*a*c))
    sol2 = 2*c / (-b + np.sqrt(b**2 - 4*a*c))
    return sol1, sol2

In [ ]:
print( quad_solve1(0.001, 1000, 0.001) )
print( quad_solve2(0.001, 1000, 0.001) )

We can see that the `quad_solve1` is a poor estimate of the first root but a good estimate of the second root, while `quad_solve2` has the opposite. Why is this the case? Could you construct a single function that outputs good estimates of both roots?

**Example: Calculating a Derivative**

If we want to calculate a derivative of the function $f(x)$ numerically, we use the definition of a derivative:

\begin{align}
\frac{\rm{d}f}{\rm{d}x} = \rm{lim}_{\delta\rightarrow0}\frac{f(x+\delta)-f(x)}{\delta}
\end{align}

Analytically, we can take $\delta\rightarrow0$ and use chain rule to take a derivative, but numerically we cannot do this. We can get a good approximation, however, if we make $\delta$ small. Let's write a program to do this. Assume $f(x) = x(x-1)$. Analytically we know this derivative is $f^{\prime}(x) = 2x-1$.


In [ ]:
def fun(x):
    return x*(x-1)

def deriv(x, delta=1e-10):
    return (fun(x+delta) - fun(x)) / delta

In [ ]:
deriv(1, delta=1e-2)

In [ ]:
deriv(1, delta=1e-6)

In [ ]:
deriv(1, delta=1e-10)

In [ ]:
deriv(1, delta=1e-12)

In [ ]:
deriv(1, delta=1e-14)

We can see that as $\delta$ becomes smaller we get a more and more accurate answer, until we get close to the floating-point precision, at which point the precision diverges. We will discuss numerical derivatives in further detail next week.